<a href="https://colab.research.google.com/github/mukulre/Projects/blob/main/Online_Payments_Fraud_Detection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("credit card.csv")
print(data.head())

   step      type    amount     nameOrig  oldbalanceOrig  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815        170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295         21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145           181.0            0.00   
3     1  CASH_OUT    181.00   C840083671           181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720         41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  
0  M1979787155             0.0             0.0      0.0  
1  M2044282225             0.0             0.0      0.0  
2   C553264065             0.0             0.0      1.0  
3    C38997010         21182.0             0.0      1.0  
4  M1230701703             0.0             0.0      0.0  


In [ ]:
print(data.isnull().sum())

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrig    0
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
dtype: int64


In [ ]:
# Exploring transaction type
print(data.type.value_counts())

type
PAYMENT     81744
CASH_OUT    76162
CASH_IN     47297
TRANSFER    18987
DEBIT        1859
Name: count, dtype: int64


In [ ]:
type = data["type"].value_counts()
transactions = type.index
quantity = type.values

import plotly.express as px
figure = px.pie(data,
             values=quantity,
             names=transactions,hole = 0.5,
             title="Distribution of Transaction Type")
figure.show()

In [ ]:
# Checking correlation
correlation = data.corr(numeric_only=True) # Add numeric_only=True
print(correlation["isFraud"].sort_values(ascending=False))

isFraud           1.000000
amount            0.046759
oldbalanceOrig   -0.001388
newbalanceDest   -0.004584
oldbalanceDest   -0.008188
newbalanceOrig   -0.008207
step             -0.035610
Name: isFraud, dtype: float64


In [ ]:
data["type"] = data["type"].map({"CASH_OUT": 1, "PAYMENT": 2,
                                 "CASH_IN": 3, "TRANSFER": 4,
                                 "DEBIT": 5})
#data["isFraud"] = data["isFraud"].map({0: "No Fraud", 1: "Fraud"})
print(data.head())

   step  type    amount     nameOrig  oldbalanceOrig  newbalanceOrig  \
0     1   NaN   9839.64  C1231006815        170136.0       160296.36   
1     1   NaN   1864.28  C1666544295         21249.0        19384.72   
2     1   NaN    181.00  C1305486145           181.0            0.00   
3     1   NaN    181.00   C840083671           181.0            0.00   
4     1   NaN  11668.14  C2048537720         41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest   isFraud  
0  M1979787155             0.0             0.0  No Fraud  
1  M2044282225             0.0             0.0  No Fraud  
2   C553264065             0.0             0.0     Fraud  
3    C38997010         21182.0             0.0     Fraud  
4  M1230701703             0.0             0.0  No Fraud  


In [ ]:
# splitting the data
from sklearn.model_selection import train_test_split
x = np.array(data[["type", "amount", "oldbalanceOrig", "newbalanceOrig"]]) # Changed 'oldbalanceOrg' to 'oldbalanceOrig'
y = np.array(data[["isFraud"]])

In [ ]:
# training a machine learning model
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)

# Impute NaN values with the most frequent value in ytrain
imputer = SimpleImputer(strategy='most_frequent') # Create imputer with 'most_frequent' strategy for categorical target
ytrain = imputer.fit_transform(ytrain) # Apply to ytrain
ytest = imputer.transform(ytest) # Apply to ytest (using the fitted imputer)

model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)
print(model.score(xtest, ytest))

0.9994691439946914


In [ ]:
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)
print(model.score(xtest, ytest))

0.9995576199955762


In [ ]:
# prediction
#features = [type, amount, oldbalanceOrg, newbalanceOrig]
features = np.array([[4, 9000.60, 9000.60, 0.0]])
print(model.predict(features))

['No Fraud']
